## 수도권 아파트 평균 매매 금액 (2014년 ~ 2023년)

### 수도권 아파트 평균 매매 금액 저장하기 위한 db 생성

- eda 프로젝트용 db 에 접근

### 팀 AWS db 에 접근

### 테이블 생성

- 년, 월, 시도, 평균 가격

In [15]:
# 테이블 생성하는 쿼리
'''create_table_quary = """
CREATE TABLE apart_price_montly (
    year INT,
    month INT,
    seoul_price INT,
    gyeonggi_price INT, 
    incheon_price INT,
    metropolitan_area INT
)"""

cur.execute(create_table_quary)'''


'create_table_quary = """\nCREATE TABLE apart_price_montly (\n    year INT,\n    month INT,\n    seoul_price INT,\n    gyeonggi_price INT, \n    incheon_price INT,\n    metropolitan_area INT\n)"""\n\ncur.execute(create_table_quary)'

### 데이터 가져오기

- 원하는 화면이 나올때 까지 selenium 으로 진행

- 다음에 beautifulsoup 으로 진행

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.alert import Alert
from bs4 import BeautifulSoup


options = webdriver.ChromeOptions()

url = 'https://www.reb.or.kr/r-one/portal/stat/easyStatPage.do'  # 부동산 통계정보 사이트 url
small_driver = webdriver.Chrome(service=Service('/home/yoon/ws/eda/driver/chromedriver'),
                          options=options)

small_driver.get(url)

small_driver.maximize_window()  # 전체화면 크기로 지정

driver = small_driver

# '전국주택가격동향조사' 클릭하기
survey_btn = driver.find_element(By.XPATH, '//*[@id="treeStatData"]/ul/li[1]/ul/li[2]/span/a/em')
survey_btn.click()

# '월간동향' 클릭하기
montly_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="treeStatData"]/ul/li[1]/ul/li[2]/ul/li[1]/span')))
montly_btn.click()

# '아파트' 클릭하기
apart_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="treeStatData"]/ul/li[1]/ul/li[2]/ul/li[1]/ul/li[2]/span')))
apart_btn.click()

# '매매가격' 클릭하기
price_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="treeStatData"]/ul/li[1]/ul/li[2]/ul/li[1]/ul/li[2]/ul/li[5]/span/a/em')))
price_btn.click()

# '(월) 평균매매가격_아파트' 클릭하기
montly_price_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="treeStatData"]/ul/li[1]/ul/li[2]/ul/li[1]/ul/li[2]/ul/li[5]/ul/li[1]/span/a')))
montly_price_btn.click()

# 유의사항 창 닫기 클릭하기
close_warning_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="SP1A"]/div/div[1]/button')))
close_warning_btn.click()

# '분류선택' 에서 모든 체크 해재 클릭하기
reset_check_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="clsBox"]/div[2]/span[2]/button[2]')))
reset_check_btn.click()

# 서울, 경기, 인천만 선택하기
seoul_check_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="treeClsData"]/ul/li[2]/span/span[2]')))
gyeonggi_check_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="treeClsData"]/ul/li[3]/span/span[2]')))
incheon_check_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="treeClsData"]/ul/li[4]/span/span[2]')))

seoul_check_btn.click()
gyeonggi_check_btn.click()
incheon_check_btn.click()

# '검색기간' 체크
period_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="wrttimeTypeB"]')))
period_btn.click()

# value 값이 2014인 키를 보낸다. (2014년 선택하기)
period_combo_box = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="wrttimeStartYear"]')))
combo_box_list = period_combo_box.find_elements(By.TAG_NAME, 'option')

for idx, year in enumerate(combo_box_list):
    if idx == 2:
        period_combo_box.send_keys(year.get_attribute('value'))

# '조회' 클릭하기
search_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="addStatTab"]')))
search_btn.click()

# IBSheet 경고창 해제
try:
    alert = WebDriverWait(driver, 10).until(EC.alert_is_present())
    alert.accept()  # 경고 창 닫기
except:
    print("경고 창이 나타나지 않았습니다.")

# beautifulsouop 으로 현재 페이지 html 형태로 가져오기
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')

## beautifulsoup 으로 진행

- 수도권의 아파트 매매 정보는 beautifulsoup 으로 가져온다.

### 테이블 속성

- YEAR int = year

- MONTH int  = month

- SIDO  varchar(16) = sido_name

- PRICE  int = real_price

### 각 데이터들 for문으로 가져오기

In [17]:
sido_name_panel = soup.find_all('div', class_='IBPageOne')[0]
sido_name_list = sido_name_panel.find_all('tr')[1:]

# 각 도시 이름을 먼저 가져오고
for idx, text in enumerate(sido_name_list):
    sido_name = text.get_text()[1:]  # 우선 도시 이름 가져오고

    # 가격 데이터 가져와야 함.
    price_panel = soup.find_all('div', class_='IBPageOne')[1]  # 가격 판넬
    city_index = price_panel.find_all('tr')[1:]  # 도시에 대한 인덱스만 들어가 있는 리스트

    # 날짜 데이터 가져오고
    month_panel = soup.find('div', class_='IBHeadMid')  # 기간이 들어있는 판넬
    month_list = month_panel.find_all('td')[1:129]  # 모든 기간을 리스트로 저장

    # 각 월에 대해 가격 데이터 출력
    sido_price_list = city_index[idx].find_all('td')[1:129]  # 해당 지역 가격 리스트
    for month_idx, date in enumerate(month_list):
        year = int(date.string[:4])  # 년도
        month_string = date.string
        if len(month_string) == 8:
            month = int(month_string[6])
        elif len(month_string) > 8:
            month = int(month_string[6:8])  # 월

        # 가격 데이터 처리
        each_price = sido_price_list[month_idx]  # 해당 월의 가격
        if idx == 0:  # 서울 인덱스에서
            if len(each_price.get_text()) > 9:
                price = each_price.get_text()[:9].replace(',', '')
            else:
                price = each_price.get_text().replace(',', '')
        else:
            price = each_price.get_text()[:7].replace(',', '')

        real_price = int(price) * 1000  # 차트상 단위가 천원이므로 1000을 곱해준다.
        
        print(f'{year}년 {month}월 {sido_name}시의 평균 아파트 매매 가격 : {real_price}')
    
    print('=' * 50)


2014년 1월 서울시의 평균 아파트 매매 가격 : 490995000
2014년 2월 서울시의 평균 아파트 매매 가격 : 492451000
2014년 3월 서울시의 평균 아파트 매매 가격 : 494338000
2014년 4월 서울시의 평균 아파트 매매 가격 : 494478000
2014년 5월 서울시의 평균 아파트 매매 가격 : 494042000
2014년 6월 서울시의 평균 아파트 매매 가격 : 493673000
2014년 7월 서울시의 평균 아파트 매매 가격 : 493652000
2014년 8월 서울시의 평균 아파트 매매 가격 : 494256000
2014년 9월 서울시의 평균 아파트 매매 가격 : 496425000
2014년 10월 서울시의 평균 아파트 매매 가격 : 498452000
2014년 11월 서울시의 평균 아파트 매매 가격 : 499009000
2014년 12월 서울시의 평균 아파트 매매 가격 : 498927000
2015년 1월 서울시의 평균 아파트 매매 가격 : 500454000
2015년 2월 서울시의 평균 아파트 매매 가격 : 502150000
2015년 3월 서울시의 평균 아파트 매매 가격 : 504979000
2015년 4월 서울시의 평균 아파트 매매 가격 : 508683000
2015년 5월 서울시의 평균 아파트 매매 가격 : 512004000
2015년 6월 서울시의 평균 아파트 매매 가격 : 515481000
2015년 7월 서울시의 평균 아파트 매매 가격 : 537536000
2015년 8월 서울시의 평균 아파트 매매 가격 : 540479000
2015년 9월 서울시의 평균 아파트 매매 가격 : 543833000
2015년 10월 서울시의 평균 아파트 매매 가격 : 547153000
2015년 11월 서울시의 평균 아파트 매매 가격 : 549204000
2015년 12월 서울시의 평균 아파트 매매 가격 : 551299000
2016년 1월 서울시의 평균 아파트 매매 가격 : 551346000
2016년 2월 서울시의 평균 아파

In [16]:
# 가격 데이터 가져와야 함.
price_panel = soup.find_all('div', class_='IBPageOne')[1]  # 가격 판넬
city_index = price_panel.find_all('tr')[1:]  # 도시에 대한 인덱스만 들어가 있는 리스트
# 서울 : 0, 경기 : 1, 인천 : 2
for i in range(len(city_index)):
    sido_price_list = city_index[i].find_all('td')[1:129]
    for idx, each_price in enumerate(sido_price_list):
            if i == 0:  # 서울 인덱스에서
                if len(each_price.get_text()) > 9:
                    price = each_price.get_text()[:9].replace(',', '')
                else:
                    price = each_price.get_text().replace(',', '')
            else:
                price = each_price.get_text()[:7].replace(',', '')
            real_price = int(price) * 1000  # 차트상 단위가 천원이므로 1000을 곱해준다.
            print(f'{i} 인덱스 지역의 {idx} 번째 가격 : {real_price}')


0 인덱스 지역의 0 번째 가격 : 490995000
0 인덱스 지역의 1 번째 가격 : 492451000
0 인덱스 지역의 2 번째 가격 : 494338000
0 인덱스 지역의 3 번째 가격 : 494478000
0 인덱스 지역의 4 번째 가격 : 494042000
0 인덱스 지역의 5 번째 가격 : 493673000
0 인덱스 지역의 6 번째 가격 : 493652000
0 인덱스 지역의 7 번째 가격 : 494256000
0 인덱스 지역의 8 번째 가격 : 496425000
0 인덱스 지역의 9 번째 가격 : 498452000
0 인덱스 지역의 10 번째 가격 : 499009000
0 인덱스 지역의 11 번째 가격 : 498927000
0 인덱스 지역의 12 번째 가격 : 500454000
0 인덱스 지역의 13 번째 가격 : 502150000
0 인덱스 지역의 14 번째 가격 : 504979000
0 인덱스 지역의 15 번째 가격 : 508683000
0 인덱스 지역의 16 번째 가격 : 512004000
0 인덱스 지역의 17 번째 가격 : 515481000
0 인덱스 지역의 18 번째 가격 : 537536000
0 인덱스 지역의 19 번째 가격 : 540479000
0 인덱스 지역의 20 번째 가격 : 543833000
0 인덱스 지역의 21 번째 가격 : 547153000
0 인덱스 지역의 22 번째 가격 : 549204000
0 인덱스 지역의 23 번째 가격 : 551299000
0 인덱스 지역의 24 번째 가격 : 551346000
0 인덱스 지역의 25 번째 가격 : 550462000
0 인덱스 지역의 26 번째 가격 : 550313000
0 인덱스 지역의 27 번째 가격 : 550936000
0 인덱스 지역의 28 번째 가격 : 552227000
0 인덱스 지역의 29 번째 가격 : 553603000
0 인덱스 지역의 30 번째 가격 : 555720000
0 인덱스 지역의 31 번째 가격 : 558017000
0 인덱스 지역의 32 번째 가격

"sido_price_list = city_index[0].find_all('td')[1:129]\nsido_price_list"

### 데이터 베이스에 insert 하기

In [20]:
sql_insert_query = """
INSERT INTO APARTMENT_AVG_PRICE_SIDO_MONTHLY (YEAR, MONTH, SIDO, PRICE)
VALUES (%s, %s, %s, %s)
"""

sido_name_panel = soup.find_all('div', class_='IBPageOne')[0]
sido_name_list = sido_name_panel.find_all('tr')[1:]

# 각 도시 이름을 먼저 가져오고
for idx, text in enumerate(sido_name_list):
    sido_name = text.get_text()[1:]  # 우선 도시 이름 가져오고

    # 가격 데이터 가져와야 함.
    price_panel = soup.find_all('div', class_='IBPageOne')[1]  # 가격 판넬
    city_index = price_panel.find_all('tr')[1:]  # 도시에 대한 인덱스만 들어가 있는 리스트

    # 날짜 데이터 가져오고
    month_panel = soup.find('div', class_='IBHeadMid')  # 기간이 들어있는 판넬
    month_list = month_panel.find_all('td')[1:129]  # 모든 기간을 리스트로 저장

    # 각 월에 대해 가격 데이터 출력
    sido_price_list = city_index[idx].find_all('td')[1:129]  # 해당 지역 가격 리스트
    for month_idx, date in enumerate(month_list):
        year = int(date.string[:4])  # 년도
        month_string = date.string
        if len(month_string) == 8:
            month = int(month_string[6])
        elif len(month_string) > 8:
            month = int(month_string[6:8])  # 월

        # 가격 데이터 처리
        each_price = sido_price_list[month_idx]  # 해당 월의 가격
        if idx == 0:  # 서울 인덱스에서
            if len(each_price.get_text()) > 9:
                price = each_price.get_text()[:9].replace(',', '')
            else:
                price = each_price.get_text().replace(',', '')
        else:
            price = each_price.get_text()[:7].replace(',', '')

        real_price = int(price) * 1000  # 차트상 단위가 천원이므로 1000을 곱해준다.
        
        # 테이블에 삽입하기
        cur.execute(sql_insert_query, (year, month, sido_name, real_price))

remote.commit()

### '수도권' 평균 가격 insert

- 각 날짜의 서울, 경기, 인천의 평균 가격을 내서

- '수도권' 이라는 SIDO 컬럼과 함께 추가하기

- 둘 다 사용 가능

In [21]:
sql_insert_query = """
INSERT INTO APARTMENT_AVG_PRICE_SIDO_MONTHLY (YEAR, MONTH, SIDO, PRICE)
VALUES (%s, %s, %s, %s)
"""

sido_name_panel = soup.find_all('div', class_='IBPageOne')[0]
sido_name_list = sido_name_panel.find_all('tr')[1:]

# 각 도시 이름을 먼저 가져오고
prices_by_month = {}  # 날짜별 가격을 저장할 딕셔너리

for idx, text in enumerate(sido_name_list):
    sido_name = text.get_text()[1:]  # 우선 도시 이름 가져오고

    # 가격 데이터 가져와야 함.
    price_panel = soup.find_all('div', class_='IBPageOne')[1]  # 가격 판넬
    city_index = price_panel.find_all('tr')[1:]  # 도시에 대한 인덱스만 들어가 있는 리스트

    # 날짜 데이터 가져오고
    month_panel = soup.find('div', class_='IBHeadMid')  # 기간이 들어있는 판넬
    month_list = month_panel.find_all('td')[1:129]  # 모든 기간을 리스트로 저장

    # 각 월에 대해 가격 데이터 출력
    sido_price_list = city_index[idx].find_all('td')[1:129]  # 해당 지역 가격 리스트
    for month_idx, date in enumerate(month_list):
        year = int(date.string[:4])  # 년도
        month_string = date.string
        if len(month_string) == 8:
            month = int(month_string[6])
        elif len(month_string) > 8:
            month = int(month_string[6:8])  # 월

        # 가격 데이터 처리
        each_price = sido_price_list[month_idx]  # 해당 월의 가격
        if idx == 0:  # 서울 인덱스에서
            if len(each_price.get_text()) > 9:
                price = each_price.get_text()[:9].replace(',', '')
            else:
                price = each_price.get_text().replace(',', '')
        else:
            price = each_price.get_text()[:7].replace(',', '')

        real_price = int(price) * 1000  # 차트상 단위가 천원이므로 1000을 곱해준다.

        # 날짜별로 가격 저장
        if (year, month) not in prices_by_month:
            prices_by_month[(year, month)] = []
        prices_by_month[(year, month)].append(real_price)

# 평균 가격을 테이블에 삽입하기
for (year, month), prices in prices_by_month.items():
    average_price = sum(prices) / len(prices) if prices else 0
    cur.execute(sql_insert_query, (year, month, '수도권', average_price))

remote.commit()


In [28]:
sql_insert_query = """
INSERT INTO APARTMENT_AVG_PRICE_SIDO_MONTHLY (YEAR, MONTH, SIDO, PRICE)
VALUES (%s, %s, %s, %s)
"""

sido_name_panel = soup.find_all('div', class_='IBPageOne')[0]
sido_name_list = sido_name_panel.find_all('tr')[1:]

# 각 도시 이름을 먼저 가져오고
prices_by_month = {'서울': {}, '경기': {}, '인천': {}}  # 도시별 날짜 가격을 저장할 딕셔너리

for idx, text in enumerate(sido_name_list):
    sido_name = text.get_text()[1:]  # 도시 이름 가져오기

    # 가격 데이터 가져와야 함.
    price_panel = soup.find_all('div', class_='IBPageOne')[1]  # 가격 판넬
    city_index = price_panel.find_all('tr')[1:]  # 도시에 대한 인덱스만 들어가 있는 리스트

    # 날짜 데이터 가져오고
    month_panel = soup.find('div', class_='IBHeadMid')  # 기간이 들어있는 판넬
    month_list = month_panel.find_all('td')[1:129]  # 모든 기간을 리스트로 저장

    # 각 월에 대해 가격 데이터 출력
    sido_price_list = city_index[idx].find_all('td')[1:129]  # 해당 지역 가격 리스트
    for month_idx, date in enumerate(month_list):
        year = int(date.string[:4])  # 년도
        month_string = date.string
        if len(month_string) == 8:
            month = int(month_string[6])
        elif len(month_string) > 8:
            month = int(month_string[6:8])  # 월

        # 가격 데이터 처리
        each_price = sido_price_list[month_idx]  # 해당 월의 가격
        if idx == 0:  # 서울 인덱스에서
            if len(each_price.get_text()) > 9:
                price = each_price.get_text()[:9].replace(',', '')
            else:
                price = each_price.get_text().replace(',', '')
        else:
            price = each_price.get_text()[:7].replace(',', '')

        real_price = int(price) * 1000  # 차트상 단위가 천원이므로 1000을 곱해준다.

        # 날짜별로 가격 저장
        if (year, month) not in prices_by_month[sido_name]:
            prices_by_month[sido_name][(year, month)] = real_price

# 평균 가격을 테이블에 삽입하기
for year_month_key in prices_by_month['서울'].keys():
    year, month = year_month_key
    seoul_price = prices_by_month['서울'][year_month_key]
    gyeonggi_price = prices_by_month['경기'][year_month_key]
    incheon_price = prices_by_month['인천'][year_month_key]

    # 서울, 경기, 인천의 가격 삽입
    cur.execute(sql_insert_query, (year, month, '서울', seoul_price))
    cur.execute(sql_insert_query, (year, month, '경기', gyeonggi_price))
    cur.execute(sql_insert_query, (year, month, '인천', incheon_price))

    # 평균 가격 계산 및 삽입
    average_price = (seoul_price + gyeonggi_price + incheon_price) / 3
    cur.execute(sql_insert_query, (year, month, '수도권', average_price))

remote.commit()


### 모두 종료

In [29]:
driver.close()
cur.close()
remote.close()